# Carga de datos

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_auc_score

In [2]:
to_eval = pd.read_csv("./data/test.csv")
train = pd.read_csv("./data/train.csv")

# Data Eng

In [3]:
to_eval_ohe = pd.get_dummies(to_eval[['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']], 
                           columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], 
                           drop_first=False)

to_eval_ids = to_eval['id']
to_eval = to_eval.drop(['person_home_ownership', 'loan_intent', 'loan_grade', 'id', 'cb_person_default_on_file'], axis=1)
to_eval = pd.concat([to_eval, to_eval_ohe], axis=1)

train_ohe = pd.get_dummies(train[['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']], 
                           columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], 
                           drop_first=False)

train = train.drop(['person_home_ownership', 'loan_intent', 'loan_grade', 'id', 'cb_person_default_on_file'], axis=1)
train = pd.concat([train, train_ohe], axis=1)

y = train['loan_status']
X = train.drop(columns=['loan_status'])

bool_cols = X.select_dtypes(include='bool').columns
X[bool_cols] = X[bool_cols].astype(int)

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

X_interactions = poly.fit_transform(X)

interaction_feature_names = poly.get_feature_names_out(input_features=X.columns)

X_interactions_df = pd.DataFrame(X_interactions, columns=interaction_feature_names)

X = pd.concat([X, X_interactions_df], axis=1)
X = X.loc[:, X.apply(pd.Series.nunique) > 1]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=140902)

In [4]:
X

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,...,loan_grade_C cb_person_default_on_file_N,loan_grade_C cb_person_default_on_file_Y,loan_grade_D cb_person_default_on_file_N,loan_grade_D cb_person_default_on_file_Y,loan_grade_E cb_person_default_on_file_N,loan_grade_E cb_person_default_on_file_Y,loan_grade_F cb_person_default_on_file_N,loan_grade_F cb_person_default_on_file_Y,loan_grade_G cb_person_default_on_file_N,loan_grade_G cb_person_default_on_file_Y
0,37,35000,0.0,6000,11.49,0.17,14,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22,56000,6.0,4000,13.35,0.07,2,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29,28800,8.0,6000,8.90,0.21,10,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30,70000,14.0,12000,11.11,0.17,5,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22,60000,2.0,6000,6.92,0.10,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,34,120000,5.0,25000,15.95,0.21,10,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
58641,28,28800,0.0,10000,12.73,0.35,8,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58642,23,44000,7.0,6800,16.00,0.15,2,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58643,22,30000,2.0,5000,8.90,0.17,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X_interactions_df

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,...,loan_grade_E loan_grade_F,loan_grade_E loan_grade_G,loan_grade_E cb_person_default_on_file_N,loan_grade_E cb_person_default_on_file_Y,loan_grade_F loan_grade_G,loan_grade_F cb_person_default_on_file_N,loan_grade_F cb_person_default_on_file_Y,loan_grade_G cb_person_default_on_file_N,loan_grade_G cb_person_default_on_file_Y,cb_person_default_on_file_N cb_person_default_on_file_Y
0,37.0,35000.0,0.0,6000.0,11.49,0.17,14.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22.0,56000.0,6.0,4000.0,13.35,0.07,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29.0,28800.0,8.0,6000.0,8.90,0.21,10.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30.0,70000.0,14.0,12000.0,11.11,0.17,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22.0,60000.0,2.0,6000.0,6.92,0.10,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,34.0,120000.0,5.0,25000.0,15.95,0.21,10.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58641,28.0,28800.0,0.0,10000.0,12.73,0.35,8.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58642,23.0,44000.0,7.0,6800.0,16.00,0.15,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58643,22.0,30000.0,2.0,5000.0,8.90,0.17,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
X_train.dtypes.value_counts()

float64    310
int32       19
int64        4
Name: count, dtype: int64

# Train and validate model

In [7]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

auc_roc = roc_auc_score(y_val, y_pred)
print(f"AUC-ROC Score: {auc_roc:.4f}")

AttributeError: 'DataFrame' object has no attribute 'dtype'

# Generate submission.csv

In [6]:
to_eval_pred = model.predict_proba(to_eval)[:, 1]

submission = pd.DataFrame({
    'ID': to_eval_ids,
    'prediction': to_eval_pred
})

submission.to_csv("submission.csv", index=False)